In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re

In [2]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [3]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
train_bur = pd.read_csv(DATA_DIR+'train_bureau.csv')
test_bur = pd.read_csv(DATA_DIR+'test_bureau.csv')

In [4]:
bur_df = pd.concat((train_bur,test_bur),axis=0)

In [5]:
train_bur.shape,train.shape,test_bur.shape,bur_df.shape

((560844, 25), (128655, 26), (64019, 25), (624863, 25))

##### date conversion and extracting payment hist start and end date

In [6]:
train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])

bur_df.sort_values(['ID','DATE-REPORTED'],ascending=[True,False],inplace=True)
bur_df = bur_df.drop_duplicates(['ID','DISBURSED-DT','CONTRIBUTOR-TYPE','DISBURSED-AMT/HIGH CREDIT'],keep='first')
bur_df['min_reported_date'] = bur_df['REPORTED DATE - HIST'].str[:-1].fillna('').str.split(',').apply(min).replace('',np.NaN)
bur_df['max_reported_date'] = bur_df['REPORTED DATE - HIST'].str[:-1].fillna('').str.split(',').apply(max).replace('',np.NaN)
bur_df['DISBURSED-DT'] = pd.to_datetime(bur_df['DISBURSED-DT'])
bur_df['CLOSE-DT'] = pd.to_datetime(bur_df['CLOSE-DT'],errors='coerce')
bur_df['LAST-PAYMENT-DATE'] = pd.to_datetime(bur_df['LAST-PAYMENT-DATE'],errors='coerce')
bur_df['max_reported_date'] = pd.to_datetime(bur_df['max_reported_date'],format='%Y%m%d',errors='coerce')
bur_df['min_reported_date'] = pd.to_datetime(bur_df['min_reported_date'],format='%Y%m%d',errors='coerce')

In [7]:
bur_df.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,min_reported_date,max_reported_date
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaT,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN,2019-08-31,2020-01-31
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaT,2019-12-19,NaN,"44,000","1,405/Monthly","20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0,2018-03-31,2019-12-31
6,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-11-15,NaT,2019-12-15,NaN,"5,00,000","7,934/Monthly","4,43,769",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",000000000000000000000000000000000000000000,"443769,448365,453134,457687,462320,466802,4712...",",,,,,,,,,,,,,,",",,,,,,,,,,,,,,",84.0,2018-11-30,2019-12-31
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaT,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,2018-03-31,2018-04-30
5,1,False,PRIMARY,Credit Card,NAB,2018-04-30,Individual,Closed,2018-01-11,2018-03-13,NaT,"50,000",0,NaN,0,NaN,NaN,0.0,Standard,"20180331,20180228,20180131,",000000000,"24650,17300,0,",",,,",",,,,",NaN,2018-01-31,2018-03-31


#### correcting the payment hist strings

###### when the report date is not present, dpd hist is always missing

In [8]:
bur_df['DPD - HIST'].isnull().sum(),bur_df['REPORTED DATE - HIST'].isnull().sum()

(8516, 8351)

In [9]:
bur_df['dpd_str_len'] = 3*bur_df['REPORTED DATE - HIST'].str[:-1].str.split(',').fillna('').apply(len)

In [10]:
%%time
out=[]
for i,x in bur_df.iterrows():
    if not ((str(x['DPD - HIST']).find('E')>-1) | (type(x['DPD - HIST'])==float)):
        out.append('0'*(x['dpd_str_len']-len(x['DPD - HIST']))+x['DPD - HIST'])
    else:
        out.append(x['DPD - HIST'])

CPU times: user 46.3 s, sys: 120 ms, total: 46.4 s
Wall time: 46.4 s


In [11]:
bur_df['dpd_string'] = out

#### correcting the amount error 

In [12]:
for col in ['OVERDUE-AMT', 'DISBURSED-AMT/HIGH CREDIT','CURRENT-BAL']:
    amt = []
    for x in bur_df[col]:
        if type(x)!=float:
            x = x.split(',')
            if len(x)>1:
                x[1] = '0'+x[1] if len(x[1])==1 else x[1]
            x = ','.join(x)
        amt.append(x)
    bur_df['corrected'+col] = amt    

In [13]:
amount_cols = ['CREDIT-LIMIT/SANC AMT','correctedDISBURSED-AMT/HIGH CREDIT', 'INSTALLMENT-AMT',
               'correctedCURRENT-BAL','correctedOVERDUE-AMT']
for col in amount_cols:
    bur_df[col] = bur_df[col].apply(lambda x:''.join(re.findall('[0-9]+',x)) if str(x)!='nan' else x).astype('float')    

In [24]:
# amount_error = pd.read_csv('amount_systematic_error.csv')
# amount_error['DISBURSED-DT'] = pd.to_datetime(amount_error['DISBURSED-DT'])

In [25]:
# amount_error.head()

In [14]:
bur_df.head()

,ID,SELF-INDICATOR,MATCH-TYPE,ACCT-TYPE,CONTRIBUTOR-TYPE,DATE-REPORTED,OWNERSHIP-IND,ACCOUNT-STATUS,DISBURSED-DT,CLOSE-DT,LAST-PAYMENT-DATE,CREDIT-LIMIT/SANC AMT,DISBURSED-AMT/HIGH CREDIT,INSTALLMENT-AMT,CURRENT-BAL,INSTALLMENT-FREQUENCY,OVERDUE-AMT,WRITE-OFF-AMT,ASSET_CLASS,REPORTED DATE - HIST,DPD - HIST,CUR BAL - HIST,AMT OVERDUE - HIST,AMT PAID - HIST,TENURE,min_reported_date,max_reported_date,dpd_str_len,dpd_string,correctedOVERDUE-AMT,correctedDISBURSED-AMT/HIGH CREDIT,correctedCURRENT-BAL
2,1,True,PRIMARY,Tractor Loan,NBF,2020-01-31,Individual,Active,2019-08-30,NaT,NaT,NaN,"1,45,000",NaN,"1,16,087",NaN,0,0.0,NaN,"20200131,20191231,20191130,20191031,20190930,2...",000000000000000000,"116087,116087,145000,145000,145000,145000,","0,0,0,0,0,0,",",,,,,,",NaN,2019-08-31,2020-01-31,18,000000000000000000,0.0,145000.0,116087.0
1,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-03-19,NaT,2019-12-19,NaN,"44,000",1405.0,"20,797",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",0000000000000000000000000000000000000000000000...,"20797,21988,23174,24341,25504,26648,27780,2891...",",,,,,,,,,,,,,,,,,,,,1452,,",",,,,,,,,,,,,,,,,,,,,,,",36.0,2018-03-31,2019-12-31,66,0000000000000000000000000000000000000000000000...,NaN,44000.0,20797.0
6,1,False,PRIMARY,Auto Loan (Personal),NAB,2019-12-31,Individual,Active,2018-11-15,NaT,2019-12-15,NaN,"5,00,000",7934.0,"4,43,769",F03,NaN,0.0,Standard,"20191231,20191130,20191031,20190930,20190831,2...",000000000000000000000000000000000000000000,"443769,448365,453134,457687,462320,466802,4712...",",,,,,,,,,,,,,,",",,,,,,,,,,,,,,",84.0,2018-11-30,2019-12-31,42,000000000000000000000000000000000000000000,NaN,500000.0,443769.0
0,1,False,PRIMARY,Overdraft,NAB,2018-04-30,Individual,Delinquent,2015-10-05,NaT,2018-02-27,NaN,"37,352",NaN,"37,873",NaN,"37,873",0.0,Standard,"20180430,20180331,",030000,"37873,12820,","37873,,",",,",NaN,2018-03-31,2018-04-30,6,030000,37873.0,37352.0,37873.0
5,1,False,PRIMARY,Credit Card,NAB,2018-04-30,Individual,Closed,2018-01-11,2018-03-13,NaT,50000.0,0,NaN,0,NaN,NaN,0.0,Standard,"20180331,20180228,20180131,",000000000,"24650,17300,0,",",,,",",,,,",NaN,2018-01-31,2018-03-31,9,000000000,NaN,0.0,0.0


In [15]:
bur_df.to_pickle(DATA_DIR+'bureau_data.pkl')

### use a combination of max/min report data, close and disb date to find the loans disbursed during the tenor of the LTFS loan